In [ ]:
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 3.00 MiB/s, done.
Resolving deltas: 100% (871/871), done.


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


In [ ]:
# runtime 100s
!cd llama2.c && python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:22<00:00, 71.3MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [ ]:
#!cd llama2.c && python tinystories.py train_vocab --vocab_size=361
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok361/tok361.vocab
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok361/tok361.model

#!cd llama2.c && python tinystories.py pretokenize --vocab_size=361
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok361/tok361.tar.gz
!cd ./llama2.c/data && tar -xf tok361.tar.gz

In [ ]:
# change bfloat16 to float16 on line 72
!sed -i "72s/.*/dtype = 'float16'  # float32|bfloat16|float16/" ./llama2.c/train.py

In [33]:
# runtime ~ 2 mins
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=361 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=1 --max_iters=1 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 361
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 1
Overriding: max_iters = 1
tokens per iteration will be: 32,768
breaks down as: 4 grad accum steps * 1 processes * 32 batch size * 256 max seq len
Initializing a new model from scratch
num decayed parameter tensors: 36, with 967,808 parameters
num non-decayed parameter tensors: 11, with 1,408 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 5.9283, val loss 5.9287
0 | loss 5.9330 | lr 0.000000e+00 | 3368.74ms | mfu -100.00%
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 1: train loss 5.9283, val loss 5.9287
saving checkpoint to out
wrote 

In [ ]:
!cd llama2.c && gcc -O3 -o run run.c -lm
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok361.model

In [34]:
!cd ./llama2.c && ./run out/model.bin -z data/tok361.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time E<unk>II9M g——l-L-&‘ppe`6)ib wJ!Dn~ ‘!-<unk>L'.0rNKélVU""â™cggmkzIEoQ\?`LL4XXJ`7]11\/L??!p9Eqs
</s>
%VV.;TEfDD90zz88aW5#S,
achieved tok/s: 812.734082
